# Reflection

In the context of LLM agent building, reflection refers to the process of prompting an LLM to observe its past steps (along with potential observations from tools/the environment) to assess the queality of the chosen actions. This is then used downstream for things like re-planning, search, or evaluation.

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the secret key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Reflection"

In [7]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    max_tokens=32768
)
generate = prompt | llm

In [8]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

Title: The Relevance of The Little Prince in Modern Childhood

The Little Prince, a novella by Antoine de Saint-Exupéry, has been a beloved classic for generations. Despite being written in the mid-20th century, the story remains highly relevant in modern childhood. Its enduring appeal can be attributed to the universal themes it explores, such as the importance of friendship, love, and the search for meaning in life.

Firstly, The Little Prince teaches children about the value of genuine friendships. The story revolves around the little prince's journey to different planets, where he encounters various grown-ups who embody different aspects of adult absurdity. The prince eventually finds a true friend in the fox, who teaches him that "one sees clearly only with the heart. Anything essential is invisible to the eyes." This lesson is particularly relevant in today's world, where children are often pressured to prioritize material possessions and superficial relationships over meaningful

## Reflect

In [9]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [11]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Essay critique and recommendations:

Title: The Timeless Relevance of The Little Prince in Modern Childhood

Strengths:
- Clear structure with an engaging introduction, well-developed main body, and a strong conclusion.
- Solid understanding of the themes presented in The Little Prince.
- Effective use of quotes to support arguments.
- Good grasp of the relevance of the story in modern childhood.

Areas for improvement:

1. Length: The essay is well-written but relatively brief. Consider expanding the essay to provide a more in-depth analysis of the themes and their relevance to modern childhood. Aim for a minimum of 500 words to allow for a more comprehensive exploration of the topics.

2. Depth: While the essay touches upon the importance of friendship, love, and self-discovery, there is room to delve deeper into each theme. For example, when discussing friendship, consider including specific examples from the book that illustrate the contrast between superficial relationships and ge

In [12]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

Title: Embracing The Little Prince's Wisdom in Modern Childhood: A Contemporary Perspective

Introduction

In the vast landscape of children's literature, Antoine de Saint-Exupéry's The Little Prince stands as a timeless classic that continues to captivate young readers. Its enduring appeal can be attributed to the universal themes it explores, such as the importance of genuine friendships, love, and self-discovery. This essay delves into these themes and examines their relevance in modern childhood, offering insights and recommendations for parents, educators, and children alike.

The Value of Genuine Friendships in the Digital Age

One of the most prominent themes in The Little Prince is the significance of genuine friendships. Throughout the story, the little prince encounters various grown-ups who fail to understand the true essence of friendship. It is only when he meets the fox that he learns about the power of meaningful connections, with the fox teaching him that "you become re

## Define graph

In [16]:
from typing import List, Sequence
from langgraph.graph import END, MessageGraph

async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})

async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    
    return HumanMessage(content=res.content)

builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()


In [17]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the tipicality of The Little Prince and its message in modern life"
        )
    ]
):
    print(event)
    print("---")



{'generate': AIMessage(content='Title: The Relevance of The Little Prince\'s Message in Modern Life\n\nIntroduction:\nAntoine de Saint-Exupéry\'s "The Little Prince" is a novella that has captured the hearts of millions of readers worldwide since its publication in 1943. The story, which follows a young prince traveling through various planets, is chock-full of timeless life lessons and philosophies that remain relevant even in today\'s modern world. This essay will explore the typicality of The Little Prince and its message in modern life.\n\nBody Paragraph 1: The Typicality of The Little Prince\nAt first glance, The Little Prince may seem like a whimsical tale about a young boy\'s adventures in space. However, upon closer examination, it becomes clear that the novella is a reflection of the human condition, making it highly typical. The characters that the Little Prince encounters on each planet, such as the vain king, the drunkard, the businessman, and the lamplighter, represent var

In [18]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Generate an essay on the tipicality of The Little Prince and its message in modern life

================================== Ai Message ==================================

Title: The Relevance of The Little Prince's Message in Modern Life

Introduction:
Antoine de Saint-Exupéry's "The Little Prince" is a novella that has captured the hearts of millions of readers worldwide since its publication in 1943. The story, which follows a young prince traveling through various planets, is chock-full of timeless life lessons and philosophies that remain relevant even in today's modern world. This essay will explore the typicality of The Little Prince and its message in modern life.

Body Paragraph 1: The Typicality of The Little Prince
At first glance, The Little Prince may seem like a whimsical tale about a young boy's adventures in space. However, upon closer examination, it becomes clear that the novella is a ref